In [2]:
#importing libraries
import re
import pickle
import ast

import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [ ]:
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
# loading the dataset
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [ ]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [ ]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [ ]:
credits.columns

Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')

In [ ]:
movies = movies.merge(credits, on = "title")

In [ ]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

In [ ]:
#Relevant features are:
# movie_id
# genres
# keywords
# overview
# title
# cast
# crew

In [ ]:
# Target dataset: id, title, tags

In [ ]:
df = movies[['movie_id','title','genres','keywords','overview','cast','crew']]
df

,movie_id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...,...,...,...
4804,9367,El Mariachi,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 5616, ""name"": ""united states\u2013mexi...",El Mariachi just wants to play his guitar and ...,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4805,72766,Newlyweds,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",[],A newlywed couple's honeymoon is upended by th...,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4806,231617,"Signed, Sealed, Delivered","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic...","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4807,126186,Shanghai Calling,[],[],When ambitious New York attorney Sam is sent t...,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4809 non-null   int64 
 1   title     4809 non-null   object
 2   genres    4809 non-null   object
 3   keywords  4809 non-null   object
 4   overview  4806 non-null   object
 5   cast      4809 non-null   object
 6   crew      4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 263.1+ KB


In [ ]:
df.dropna(inplace = True)

In [ ]:
df.isnull().value_counts()

movie_id  title  genres  keywords  overview  cast   crew 
False     False  False   False     False     False  False    4806
Name: count, dtype: int64

In [ ]:
# genres

In [ ]:
df['genres']

0       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1       [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
3       [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
                              ...                        
4804    [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4805    [{"id": 35, "name": "Comedy"}, {"id": 10749, "...
4806    [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...
4807                                                   []
4808                  [{"id": 99, "name": "Documentary"}]
Name: genres, Length: 4806, dtype: object

In [ ]:
ast.literal_eval(df['genres'][0])

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [ ]:
for i in ast.literal_eval(df['genres'][0]):
    print(i['name'])

Action
Adventure
Fantasy
Science Fiction


In [ ]:
def get_genres(genre):
    """Cleans the feature genre and returs only the names"""
    genres = []
    for i in ast.literal_eval(genre):
        genres.append(i['name'])

    return genres

In [ ]:
get_genres(df['genres'][0])

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [ ]:
df['genres'] = df['genres'].apply(get_genres)
df['genres']

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4804                        [Action, Crime, Thriller]
4805                                [Comedy, Romance]
4806               [Comedy, Drama, Romance, TV Movie]
4807                                               []
4808                                    [Documentary]
Name: genres, Length: 4806, dtype: object

In [ ]:
#keywords

In [ ]:
df['keywords']

0       [{"id": 1463, "name": "culture clash"}, {"id":...
1       [{"id": 270, "name": "ocean"}, {"id": 726, "na...
2       [{"id": 470, "name": "spy"}, {"id": 818, "name...
3       [{"id": 849, "name": "dc comics"}, {"id": 853,...
4       [{"id": 818, "name": "based on novel"}, {"id":...
                              ...                        
4804    [{"id": 5616, "name": "united states\u2013mexi...
4805                                                   []
4806    [{"id": 248, "name": "date"}, {"id": 699, "nam...
4807                                                   []
4808    [{"id": 1523, "name": "obsession"}, {"id": 224...
Name: keywords, Length: 4806, dtype: object

In [ ]:
df['keywords'][0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [ ]:
def get_keywords(text):
    """To clean the feature keywords"""
    keywords = []
    for i in ast.literal_eval(text):
        keywords.append(i['name'])

    return keywords

In [ ]:
get_keywords(df['keywords'][0])

['culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [ ]:
df['keywords'] = df['keywords'].apply(get_keywords)
df['keywords']

0       [culture clash, future, space war, space colon...
1       [ocean, drug abuse, exotic island, east india ...
2       [spy, based on novel, secret agent, sequel, mi...
3       [dc comics, crime fighter, terrorist, secret i...
4       [based on novel, mars, medallion, space travel...
                              ...                        
4804    [united states–mexico barrier, legs, arms, pap...
4805                                                   []
4806    [date, love at first sight, narration, investi...
4807                                                   []
4808            [obsession, camcorder, crush, dream girl]
Name: keywords, Length: 4806, dtype: object

In [ ]:
#overview

In [ ]:
df['overview']

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4804    El Mariachi just wants to play his guitar and ...
4805    A newlywed couple's honeymoon is upended by th...
4806    "Signed, Sealed, Delivered" introduces a dedic...
4807    When ambitious New York attorney Sam is sent t...
4808    Ever since the second grade when he first saw ...
Name: overview, Length: 4806, dtype: object

In [ ]:
df['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [ ]:
df['overview'][0].split()

['In',
 'the',
 '22nd',
 'century,',
 'a',
 'paraplegic',
 'Marine',
 'is',
 'dispatched',
 'to',
 'the',
 'moon',
 'Pandora',
 'on',
 'a',
 'unique',
 'mission,',
 'but',
 'becomes',
 'torn',
 'between',
 'following',
 'orders',
 'and',
 'protecting',
 'an',
 'alien',
 'civilization.']

In [ ]:
df['overview'] = df['overview'].apply(lambda x : x.split())

In [ ]:
df['overview']

0       [In, the, 22nd, century,, a, paraplegic, Marin...
1       [Captain, Barbossa,, long, believed, to, be, d...
2       [A, cryptic, message, from, Bond’s, past, send...
3       [Following, the, death, of, District, Attorney...
4       [John, Carter, is, a, war-weary,, former, mili...
                              ...                        
4804    [El, Mariachi, just, wants, to, play, his, gui...
4805    [A, newlywed, couple's, honeymoon, is, upended...
4806    ["Signed,, Sealed,, Delivered", introduces, a,...
4807    [When, ambitious, New, York, attorney, Sam, is...
4808    [Ever, since, the, second, grade, when, he, fi...
Name: overview, Length: 4806, dtype: object

In [ ]:
#cast

In [ ]:
df['cast']

0       [{"cast_id": 242, "character": "Jake Sully", "...
1       [{"cast_id": 4, "character": "Captain Jack Spa...
2       [{"cast_id": 1, "character": "James Bond", "cr...
3       [{"cast_id": 2, "character": "Bruce Wayne / Ba...
4       [{"cast_id": 5, "character": "John Carter", "c...
                              ...                        
4804    [{"cast_id": 1, "character": "El Mariachi", "c...
4805    [{"cast_id": 1, "character": "Buzzy", "credit_...
4806    [{"cast_id": 8, "character": "Oliver O\u2019To...
4807    [{"cast_id": 3, "character": "Sam", "credit_id...
4808    [{"cast_id": 3, "character": "Herself", "credi...
Name: cast, Length: 4806, dtype: object

In [ ]:
df['cast'][0]

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [ ]:
ast.literal_eval(df['cast'][0])[0].keys()

dict_keys(['cast_id', 'character', 'credit_id', 'gender', 'id', 'name', 'order'])

In [ ]:
def get_cast(text):
    cast = []
    for i in ast.literal_eval(text):
        cast.append(i['name'])

    return cast

In [ ]:
get_cast(df['cast'][0])

['Sam Worthington',
 'Zoe Saldana',
 'Sigourney Weaver',
 'Stephen Lang',
 'Michelle Rodriguez',
 'Giovanni Ribisi',
 'Joel David Moore',
 'CCH Pounder',
 'Wes Studi',
 'Laz Alonso',
 'Dileep Rao',
 'Matt Gerald',
 'Sean Anthony Moran',
 'Jason Whyte',
 'Scott Lawrence',
 'Kelly Kilgour',
 'James Patrick Pitt',
 'Sean Patrick Murphy',
 'Peter Dillon',
 'Kevin Dorman',
 'Kelson Henderson',
 'David Van Horn',
 'Jacob Tomuri',
 'Michael Blain-Rozgay',
 'Jon Curry',
 'Luke Hawker',
 'Woody Schultz',
 'Peter Mensah',
 'Sonia Yee',
 'Jahnel Curfman',
 'Ilram Choi',
 'Kyla Warren',
 'Lisa Roumain',
 'Debra Wilson',
 'Chris Mala',
 'Taylor Kibby',
 'Jodie Landau',
 'Julie Lamm',
 'Cullen B. Madden',
 'Joseph Brady Madden',
 'Frankie Torres',
 'Austin Wilson',
 'Sara Wilson',
 'Tamica Washington-Miller',
 'Lucy Briant',
 'Nathan Meister',
 'Gerry Blair',
 'Matthew Chamberlain',
 'Paul Yates',
 'Wray Wilson',
 'James Gaylyn',
 'Melvin Leno Clark III',
 'Carvon Futrell',
 'Brandon Jelkes',
 'Mica

In [ ]:
df['cast'] = df['cast'].apply(get_cast)
df['cast']

0       [Sam Worthington, Zoe Saldana, Sigourney Weave...
1       [Johnny Depp, Orlando Bloom, Keira Knightley, ...
2       [Daniel Craig, Christoph Waltz, Léa Seydoux, R...
3       [Christian Bale, Michael Caine, Gary Oldman, A...
4       [Taylor Kitsch, Lynn Collins, Samantha Morton,...
                              ...                        
4804    [Carlos Gallardo, Jaime de Hoyos, Peter Marqua...
4805    [Edward Burns, Kerry Bishé, Marsha Dietlein, C...
4806    [Eric Mabius, Kristin Booth, Crystal Lowe, Geo...
4807    [Daniel Henney, Eliza Coupe, Bill Paxton, Alan...
4808    [Drew Barrymore, Brian Herzlinger, Corey Feldm...
Name: cast, Length: 4806, dtype: object

In [ ]:
#crew

In [ ]:
df['crew']

0       [{"credit_id": "52fe48009251416c750aca23", "de...
1       [{"credit_id": "52fe4232c3a36847f800b579", "de...
2       [{"credit_id": "54805967c3a36829b5002c41", "de...
3       [{"credit_id": "52fe4781c3a36847f81398c3", "de...
4       [{"credit_id": "52fe479ac3a36847f813eaa3", "de...
                              ...                        
4804    [{"credit_id": "52fe44eec3a36847f80b280b", "de...
4805    [{"credit_id": "52fe487dc3a368484e0fb013", "de...
4806    [{"credit_id": "52fe4df3c3a36847f8275ecf", "de...
4807    [{"credit_id": "52fe4ad9c3a368484e16a36b", "de...
4808    [{"credit_id": "58ce021b9251415a390165d9", "de...
Name: crew, Length: 4806, dtype: object

In [ ]:
df['crew'][0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [ ]:
ast.literal_eval(df['crew'][0])[0].keys()

dict_keys(['credit_id', 'department', 'gender', 'id', 'job', 'name'])

In [ ]:
s = set()
for i in ast.literal_eval(df['crew'][0]):
    s.add(i['job'])

print(s)

{'Still Photographer', 'Supervising Art Director', 'Art Department Coordinator', 'Stunts', 'Modeling', 'Stunt Coordinator', 'Editor', 'Visual Effects Supervisor', 'Visual Effects Art Director', 'Assistant Art Director', 'Publicist', 'Art Department Manager', 'Casting', 'Sound Effects Editor', 'Steadicam Operator', 'Special Effects Coordinator', 'Set Costumer', 'Director of Photography', 'CG Supervisor', 'Foley', 'Director', 'Lighting Artist', 'Art Direction', 'Construction Coordinator', 'Costume Design', 'Dialect Coach', 'Visual Effects Producer', 'Animation Director', 'Production Manager', 'Sound Designer', 'Makeup Artist', 'Executive Producer', 'Conceptual Design', 'Choreographer', 'Supervising Sound Editor', 'Dialogue Editor', 'Sound Re-Recording Mixer', 'Best Boy Electric', 'Visual Effects Editor', 'Production Design', 'Set Designer', 'Lighting Technician', 'Set Decoration', 'Makeup Department Head', 'Screenplay', 'Production Supervisor', 'Original Music Composer', 'Digital Interme

In [ ]:
def get_crew(text):
    crew = set()
    for i in ast.literal_eval(text):
        if i['job'] == "Director":
            crew.add(i['name']) 

    return list(crew)

In [ ]:
get_crew(df['crew'][0])

['James Cameron']

In [ ]:
df['crew'] = df['crew'].apply(get_crew)
df['crew']

0                                [James Cameron]
1                               [Gore Verbinski]
2                                   [Sam Mendes]
3                            [Christopher Nolan]
4                               [Andrew Stanton]
                          ...                   
4804                          [Robert Rodriguez]
4805                              [Edward Burns]
4806                               [Scott Smith]
4807                               [Daniel Hsia]
4808    [Brian Herzlinger, Jon Gunn, Brett Winn]
Name: crew, Length: 4806, dtype: object

In [ ]:
df.head()

,movie_id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[In, the, 22nd, century,, a, paraplegic, Marin...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Captain, Barbossa,, long, believed, to, be, d...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]
2,206647,Spectre,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[A, cryptic, message, from, Bond’s, past, send...","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Following, the, death, of, District, Attorney...","[Christian Bale, Michael Caine, Gary Oldman, A...",[Christopher Nolan]
4,49529,John Carter,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[John, Carter, is, a, war-weary,, former, mili...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...",[Andrew Stanton]


In [ ]:
df['tags'] = df['genres'] + df['keywords'] + df['overview'] + df['cast'] + df['crew']

In [ ]:
df['tags']

0       [Action, Adventure, Fantasy, Science Fiction, ...
1       [Adventure, Fantasy, Action, ocean, drug abuse...
2       [Action, Adventure, Crime, spy, based on novel...
3       [Action, Crime, Drama, Thriller, dc comics, cr...
4       [Action, Adventure, Science Fiction, based on ...
                              ...                        
4804    [Action, Crime, Thriller, united states–mexico...
4805    [Comedy, Romance, A, newlywed, couple's, honey...
4806    [Comedy, Drama, Romance, TV Movie, date, love ...
4807    [When, ambitious, New, York, attorney, Sam, is...
4808    [Documentary, obsession, camcorder, crush, dre...
Name: tags, Length: 4806, dtype: object

In [ ]:
data = df[['movie_id', 'title', 'tags']]
data.head()

,movie_id,title,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction, ..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, ocean, drug abuse..."
2,206647,Spectre,"[Action, Adventure, Crime, spy, based on novel..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, dc comics, cr..."
4,49529,John Carter,"[Action, Adventure, Science Fiction, based on ..."


In [ ]:
ps = PorterStemmer()

def preprocess(text):
    clean_data = ""

    for i in text:
        clean_data += " " + ps.stem(i.lower())

    return clean_data

In [ ]:
preprocess(data['tags'][0])

' action adventur fantasi science fict culture clash futur space war space coloni societi space travel futurist romanc space alien tribe alien planet cgi marin soldier battl love affair anti war power rel mind and soul 3d in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. sam worthington zoe saldana sigourney weav stephen lang michelle rodriguez giovanni ribisi joel david moor cch pounder wes studi laz alonso dileep rao matt gerald sean anthony moran jason whyt scott lawr kelly kilgour james patrick pitt sean patrick murphi peter dillon kevin dorman kelson henderson david van horn jacob tomuri michael blain-rozgay jon curri luke hawk woody schultz peter mensah sonia ye jahnel curfman ilram choi kyla warren lisa roumain debra wilson chris mala taylor kibbi jodie landau julie lamm cullen b. madden joseph brady madden frankie torr austin wilson sara wilson tamica washington-mil lu

In [ ]:
data['tags'] = data['tags'].apply(preprocess)
data['tags']

0        action adventur fantasi science fict culture ...
1        adventur fantasi action ocean drug abus exoti...
2        action adventur crime spi based on novel secr...
3        action crime drama thriller dc comic crime fi...
4        action adventur science fict based on novel m...
                              ...                        
4804     action crime thriller united states–mexico ba...
4805     comedi romanc a newlyw couple' honeymoon is u...
4806     comedi drama romanc tv movi date love at firs...
4807     when ambiti new york attorney sam is sent to ...
4808     documentari obsess camcord crush dream girl e...
Name: tags, Length: 4806, dtype: object

In [ ]:
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

In [ ]:
cv

CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
cv.fit_transform(data['tags'])

<4806x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 285146 stored elements in Compressed Sparse Row format>

In [ ]:
vectors = cv.fit_transform(data['tags']).toarray()
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [ ]:
vectors.shape

(4806, 5000)

In [ ]:
cosine_similarity(vectors)

array([[1.        , 0.11277088, 0.07525767, ..., 0.03501505, 0.01904762,
        0.02279212],
       [0.11277088, 1.        , 0.06552366, ..., 0.03919644, 0.01421482,
        0.022679  ],
       [0.07525767, 0.06552366, 1.        , ..., 0.02470454, 0.05375548,
        0.        ],
       ...,
       [0.03501505, 0.03919644, 0.02470454, ..., 1.        , 0.03751612,
        0.04489127],
       [0.01904762, 0.01421482, 0.05375548, ..., 0.03751612, 1.        ,
        0.06512033],
       [0.02279212, 0.022679  , 0.        , ..., 0.04489127, 0.06512033,
        1.        ]])

In [ ]:
similarities = cosine_similarity(vectors)

In [ ]:
sorted(similarities[0], reverse = True)

[1.0,
 0.2985111570629967,
 0.2895309965199667,
 0.26814805918095336,
 0.25712345174575735,
 0.25248351833636407,
 0.24724901510764022,
 0.24423639393424634,
 0.23851391759997756,
 0.23792475099178506,
 0.23664319132398465,
 0.2309401076758503,
 0.23050478192086757,
 0.2303030303030303,
 0.22847628957518573,
 0.22792115291927592,
 0.227845518382332,
 0.22764241217547274,
 0.2274509803921569,
 0.22618371458132555,
 0.22304756293582473,
 0.22120401653500082,
 0.22110831935702663,
 0.2200568721233588,
 0.21949587390588923,
 0.21585870860091105,
 0.21566554640687685,
 0.2148148148148148,
 0.2147096128842948,
 0.21444438569900387,
 0.21444438569900387,
 0.21402142179250075,
 0.21371868349696446,
 0.2136206984748936,
 0.21316039089348393,
 0.2130032168075646,
 0.21282316919136587,
 0.2127264093913242,
 0.21259261076849462,
 0.21227460057813102,
 0.21088784746999134,
 0.21076152500022488,
 0.20986887291884124,
 0.20844250013389537,
 0.20742472018382752,
 0.2070498521970933,
 0.205305961029894

In [ ]:
list(enumerate(similarities[0]))

[(0, 1.0),
 (1, 0.11277088155713211),
 (2, 0.07525766947068777),
 (3, 0.21402142179250075),
 (4, 0.22110831935702663),
 (5, 0.18752986199344393),
 (6, 0.051320023927966744),
 (7, 0.13653865341984012),
 (8, 0.11190057698883765),
 (9, 0.20399709511366776),
 (10, 0.11322770341445959),
 (11, 0.07106690545187014),
 (12, 0.08845379626717034),
 (13, 0.1253216719980732),
 (14, 0.1301200097264711),
 (15, 0.05819143739626464),
 (16, 0.15067564803944308),
 (17, 0.11421148705787473),
 (18, 0.10690449676496976),
 (19, 0.12440104301972907),
 (20, 0.10459527207369813),
 (21, 0.06271472455890648),
 (22, 0.12221047629723021),
 (23, 0.09689627902499089),
 (24, 0.09449111825230683),
 (25, 0.20098394458765445),
 (26, 0.16510016510024766),
 (27, 0.21282316919136587),
 (28, 0.11007906440048995),
 (29, 0.06988565578146122),
 (30, 0.1610152971798826),
 (31, 0.16318634535487714),
 (32, 0.08314794192830981),
 (33, 0.10706067580626215),
 (34, 0.06158402871356007),
 (35, 0.17482935149524953),
 (36, 0.183339699405

In [ ]:
sorted(list(enumerate(similarities[0])), reverse = True)

[(4805, 0.022792115291927588),
 (4804, 0.019047619047619046),
 (4803, 0.035015048762592677),
 (4802, 0.016169041669088863),
 (4801, 0.03142696805273545),
 (4800, 0.006768974434224128),
 (4799, 0.09701425001453318),
 (4798, 0.13251783128981584),
 (4797, 0.02666666666666667),
 (4796, 0.018569533817705187),
 (4795, 0.0),
 (4794, 0.057166195047502935),
 (4793, 0.02519763153394848),
 (4792, 0.07472647177570733),
 (4791, 0.035015048762592677),
 (4790, 0.09335200560186732),
 (4789, 0.09428090415820634),
 (4788, 0.04868644955601476),
 (4787, 0.01509702712192794),
 (4786, 0.046435068876466166),
 (4785, 0.022792115291927588),
 (4784, 0.020100756305184243),
 (4783, 0.027216552697590872),
 (4782, 0.04649905549752771),
 (4781, 0.06735753140545635),
 (4780, 0.007106690545187013),
 (4779, 0.058925565098878946),
 (4778, 0.009245003270420486),
 (4777, 0.08908708063747481),
 (4776, 0.08401680504168058),
 (4775, 0.05298129428260175),
 (4774, 0.04082482904638631),
 (4773, 0.01095993248702382),
 (4772, 0.0

In [ ]:
sorted(list(enumerate(similarities[0])), reverse = True, key = lambda x : x[1])

[(0, 1.0),
 (2409, 0.2985111570629967),
 (47, 0.2895309965199667),
 (149, 0.26814805918095336),
 (65, 0.25712345174575735),
 (1920, 0.25248351833636407),
 (1321, 0.24724901510764022),
 (1537, 0.24423639393424634),
 (1082, 0.23851391759997756),
 (305, 0.23792475099178506),
 (1216, 0.23664319132398465),
 (4336, 0.2309401076758503),
 (507, 0.23050478192086757),
 (52, 0.2303030303030303),
 (72, 0.22847628957518573),
 (3675, 0.22792115291927592),
 (505, 0.227845518382332),
 (1496, 0.22764241217547274),
 (322, 0.2274509803921569),
 (1383, 0.22618371458132555),
 (4048, 0.22304756293582473),
 (278, 0.22120401653500082),
 (4, 0.22110831935702663),
 (598, 0.2200568721233588),
 (110, 0.21949587390588923),
 (158, 0.21585870860091105),
 (300, 0.21566554640687685),
 (3162, 0.2148148148148148),
 (1607, 0.2147096128842948),
 (270, 0.21444438569900387),
 (838, 0.21444438569900387),
 (3, 0.21402142179250075),
 (95, 0.21371868349696446),
 (2204, 0.2136206984748936),
 (3293, 0.21316039089348393),
 (111, 0

In [ ]:
def recommend(movie):
    index = data[data['title'] == movie].index[0]
    recommendations = sorted(list(enumerate(similarities[index])), reverse = True, key = lambda x : x[1])[1:6]

    for i in recommendations:
        print(data.iloc[i[0]]['title'])

In [ ]:
recommend("Iron Man")

Iron Man 3
Iron Man 2
The Avengers
Avengers: Age of Ultron
The Incredible Hulk


In [ ]:
recommend("Harry Potter and the Chamber of Secrets")

Harry Potter and the Philosopher's Stone
Harry Potter and the Order of the Phoenix
Harry Potter and the Prisoner of Azkaban
Harry Potter and the Half-Blood Prince
Harry Potter and the Goblet of Fire


In [ ]:
recommend("Spider-Man")

Spider-Man 2
Spider-Man 3
The Amazing Spider-Man 2
Oz: The Great and Powerful
Iron Man 2


In [ ]:
data

,movie_id,title,tags
0,19995,Avatar,action adventur fantasi science fict culture ...
1,285,Pirates of the Caribbean: At World's End,adventur fantasi action ocean drug abus exoti...
2,206647,Spectre,action adventur crime spi based on novel secr...
3,49026,The Dark Knight Rises,action crime drama thriller dc comic crime fi...
4,49529,John Carter,action adventur science fict based on novel m...
...,...,...,...
4804,9367,El Mariachi,action crime thriller united states–mexico ba...
4805,72766,Newlyweds,comedi romanc a newlyw couple' honeymoon is u...
4806,231617,"Signed, Sealed, Delivered",comedi drama romanc tv movi date love at firs...
4807,126186,Shanghai Calling,when ambiti new york attorney sam is sent to ...


In [ ]:
data.to_dict()

{'movie_id': {0: 19995,
  1: 285,
  2: 206647,
  3: 49026,
  4: 49529,
  5: 559,
  6: 38757,
  7: 99861,
  8: 767,
  9: 209112,
  10: 1452,
  11: 10764,
  12: 58,
  13: 57201,
  14: 49521,
  15: 2454,
  16: 24428,
  17: 1865,
  18: 41154,
  19: 122917,
  20: 1930,
  21: 20662,
  22: 57158,
  23: 2268,
  24: 254,
  25: 597,
  26: 271110,
  27: 44833,
  28: 135397,
  29: 37724,
  30: 558,
  31: 68721,
  32: 12155,
  33: 36668,
  34: 62211,
  35: 8373,
  36: 91314,
  37: 68728,
  38: 102382,
  39: 20526,
  40: 49013,
  41: 44912,
  42: 10193,
  43: 534,
  44: 168259,
  45: 72190,
  46: 127585,
  47: 54138,
  48: 81005,
  49: 64682,
  50: 9543,
  51: 68726,
  52: 38356,
  53: 217,
  54: 105864,
  55: 62177,
  56: 188927,
  57: 10681,
  58: 5174,
  59: 14161,
  60: 17979,
  61: 76757,
  62: 258489,
  63: 411,
  64: 246655,
  65: 155,
  66: 14160,
  67: 15512,
  68: 1726,
  69: 44826,
  70: 8487,
  71: 1735,
  72: 297761,
  73: 2698,
  74: 137113,
  75: 9804,
  76: 14869,
  77: 150540,
  78:

In [ ]:
pickle.dump(data.to_dict(), open("movie.pkl", mode = "wb"))

In [ ]:
pickle.dump(similarities, open("similarities.pkl", mode = "wb"))

In [ ]:
list(data['title'])

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [ ]:
recommend("Cars 2")

Cars
Public Enemies
Gandhi
Stripes
Hoffa


In [ ]:
recommend("Superman Returns")

Superman III
The Wolverine
Man of Steel
Superman
Superman IV: The Quest for Peace
